# Reduction of DFOSC data with CCDproc 

See here for the documentation of https://ccdproc.readthedocs.io/en/latest/reduction_toolbox.html#clean-image 

In [10]:
# import the relevant packages 
import sys
import os
import numpy as np
  
from astropy.io import fits
from astropy import units as u

import ccdproc 
from ccdproc import CCDData

from ccdproc import ImageFileCollection

Define where are the raw and the reduced data? 

In [28]:
inputdir = '/Users/lasilla/dfosc_testdata/raw/'
outputdir = '/Users/lasilla/dfosc_testdata/reduced/'

In [29]:
# if there's no output directory, make one: 
if not os.path.isdir(outputdir): 
    os.mkdir(outputdir)

We first want to sort the data according to its type (i.e. bias, flat...). The keyword in the fits header is `object`.

In [73]:
# collect the image files 
ifc = ImageFileCollection(inputdir, keywords='*')
ifc.files

['Flat_I_000002.fits',
 'Flat_I_000003.fits',
 'Flat_I_000004.fits',
 'Flat_I_000005.fits',
 'Flat_I_000006.fits',
 'Flat_I_000007.fits',
 'Flat_I_000008.fits',
 'Flat_I_000009.fits',
 'Flat_I_000010.fits',
 'Flat_I_000011.fits',
 'Flat_I_000012.fits',
 'bias_000001.fits',
 'bias_000002.fits',
 'bias_000003.fits',
 'bias_000004.fits',
 'bias_000005.fits',
 'bias_000006.fits',
 'bias_000007.fits',
 'bias_000008.fits',
 'bias_000009.fits',
 'bias_000010.fits',
 'bias_000011.fits',
 'bias_000012.fits',
 'bias_000013.fits',
 'bias_000014.fits',
 'bias_000015.fits',
 'masterflat-B-2020-01-19_even.fits',
 'masterflat-I-2020-02-02_morn.fits',
 'masterflat-R-2020-02-01_morn.fits',
 'masterflat-V-2020-01-31_morn.fits',
 'testwasp24_000002.fits',
 'testwasp24_000003.fits']

In [140]:
matches_bias = ifc.summary['object'] == 'bias'
matches_flat = ifc.summary['object'] == 'Flat_I'
matches_sci_I = (ifc.summary['object'] == 'testwasp24') & (ifc.summary['filtb'] == 'I')
bias_files = [ifc.location + file for file in np.array(ifc.files)[matches_bias]]
flat_files = [ifc.location + file for file in np.array(ifc.files)[matches_flat]] 
science_files_I = [ifc.location + file for file in np.array(ifc.files)[matches_sci_I]]


We next create the master bias frame:

In [150]:
bias_list = []
for filename in bias_files:
    print(filename)
    ccd = CCDData.read(filename, unit = u.adu)
    #this has to be fixed as the bias section does not include the whole section that will be trimmed
#     ccd = ccdproc.subtract_overscan(ccd, median=True,  overscan_axis=1, fits_section='[1:75,2112:2147]')
    ccd = ccdproc.trim_image(ccd, fits_section='[76:2106, :]')
    bias_list.append(ccd)
master_bias = ccdproc.combine(bias_list, method='median')
master_bias.write('master_bias.fits', clobber=True)

/Users/lasilla/dfosc_testdata/raw/bias_000001.fits
/Users/lasilla/dfosc_testdata/raw/bias_000002.fits
/Users/lasilla/dfosc_testdata/raw/bias_000003.fits
/Users/lasilla/dfosc_testdata/raw/bias_000004.fits
/Users/lasilla/dfosc_testdata/raw/bias_000005.fits
/Users/lasilla/dfosc_testdata/raw/bias_000006.fits
/Users/lasilla/dfosc_testdata/raw/bias_000007.fits
/Users/lasilla/dfosc_testdata/raw/bias_000008.fits
/Users/lasilla/dfosc_testdata/raw/bias_000009.fits
/Users/lasilla/dfosc_testdata/raw/bias_000010.fits
/Users/lasilla/dfosc_testdata/raw/bias_000011.fits
/Users/lasilla/dfosc_testdata/raw/bias_000012.fits


INFO:astropy:splitting each image into 1 chunks to limit memory usage to 16000000000.0 bytes.


/Users/lasilla/dfosc_testdata/raw/bias_000013.fits
/Users/lasilla/dfosc_testdata/raw/bias_000014.fits
/Users/lasilla/dfosc_testdata/raw/bias_000015.fits
INFO: splitting each image into 1 chunks to limit memory usage to 16000000000.0 bytes. [ccdproc.combiner]


Now we create the master flat

In [151]:
flat_I_list = []
for filename in flat_files:
    print(filename)
    ccd = CCDData.read(filename, unit = u.adu)
    #this has to be fixed as the bias section does not include the whole section that will be trimmed
    #ccd = ccdproc.subtract_overscan(ccd, median=True,  overscan_axis=0, fits_section='[1:966,4105:4190]')
    ccd = ccdproc.trim_image(ccd, fits_section='[76:2106, :]')
    ccd = ccdproc.subtract_bias(ccd, master_bias)
    flat_I_list.append(ccd)
master_flat_I = ccdproc.combine(flat_I_list, method='median')
master_flat_I.write('master_flat_I.fits', clobber=True)


/Users/lasilla/dfosc_testdata/raw/Flat_I_000002.fits
/Users/lasilla/dfosc_testdata/raw/Flat_I_000003.fits
/Users/lasilla/dfosc_testdata/raw/Flat_I_000004.fits
/Users/lasilla/dfosc_testdata/raw/Flat_I_000005.fits
/Users/lasilla/dfosc_testdata/raw/Flat_I_000006.fits
/Users/lasilla/dfosc_testdata/raw/Flat_I_000007.fits
/Users/lasilla/dfosc_testdata/raw/Flat_I_000008.fits
/Users/lasilla/dfosc_testdata/raw/Flat_I_000009.fits
/Users/lasilla/dfosc_testdata/raw/Flat_I_000010.fits


INFO:astropy:splitting each image into 1 chunks to limit memory usage to 16000000000.0 bytes.


/Users/lasilla/dfosc_testdata/raw/Flat_I_000011.fits
/Users/lasilla/dfosc_testdata/raw/Flat_I_000012.fits
INFO: splitting each image into 1 chunks to limit memory usage to 16000000000.0 bytes. [ccdproc.combiner]


And now we just need to correct the science files :) We also remove cosmic rays^[I didn't see any in the raw data, but better save than sorry] and scale by the exposure time such that the resulting file is in counts/s.

In [157]:
for filename in science_files_I:
    print(filename)
    savename = os.path.basename(filename)
    hdu = fits.open(filename)
    ccd = CCDData(hdu['PRIMARY'].data, header=hdu['PRIMARY'].header, unit = u.adu)
    #this has to be fixed as the bias section does not include the whole section that will be trimmed
    #ccd = ccdproc.subtract_overscan(ccd, median=True,  overscan_axis=0, fits_section='[1:966,4105:4190]')
    ccd = ccdproc.trim_image(ccd, fits_section='[76:2106, :]')
    ccd = ccdproc.subtract_bias(ccd, master_bias)
    ccd = ccdproc.flat_correct(ccd, master_flat_I)
    ccd.write('sub_'+ savename, clobber=True)
    ccd = ccdproc.cosmicray_lacosmic(ccd, sigclip=5) # remove cosmic rays
    ccd.write('cr_'+ savename, clobber=True)
    ccd.data = ccd.data.T
    exptime = ccd.header['EXPTIME']
    ccd.write('obj_'+ savename, clobber=True)

/Users/lasilla/dfosc_testdata/raw/testwasp24_000002.fits
/Users/lasilla/dfosc_testdata/raw/testwasp24_000003.fits


Now the only thing left is do to the stacking. 